In [ ]:
import numpy as np
import plotly.express as px
from numba import njit, prange, cuda
import math
import cupy as cp
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
HEIGHT = 1440*4
WIDTH = 2560*4
MAX_X = .65
MIN_X = -2.35
MAX_Y = 1.25
MIN_Y = -1.25

In [ ]:
@njit
def test_point_trap(a, b, max_iter, dist):
    x = y = 0
    d = 1e9
    for _ in range(max_iter):
        t_x = x*x - y*y + a
        t_y = 2*x*y + b
        x = t_x
        y = t_y
        d = min(d, dist(x, y))
    return d

In [ ]:
@njit
def test_point(a, b, max_iter, escaped):
    x = y = 0
    for i in range(max_iter):
        t_x = x*x - y*y + a
        t_y = 2*x*y + b
        x = t_x
        y = t_y
        if escaped(x, y):
            return i+1
    return 0

In [ ]:
@njit
def escaped(x, y):
    return x*x + y*y > 4

@njit
def scale(x, min, max):
    return (max-min)*x+min

@njit(parallel=True)
def generate_fractal(arr, escaped=None, max_iter=100, dist=None):
    func = partial(test_point, escaped=escaped) if dist is None else partial(test_point_trap, dist=dist)
    for i in prange(WIDTH):
        for j in prange(HEIGHT):
            u = scale(i/(WIDTH-1), MIN_X, MAX_X)
            v = scale(j/(HEIGHT-1), MIN_Y, MAX_Y)
            arr[j, i] = func(u, v, max_iter=max_iter)

@cuda.jit
def cuda_generate_fractal(arr, max_iter):
    j, i = cuda.grid(2)
    a = scale(i/(WIDTH-1), MIN_X, MAX_X)
    b = scale(j/(HEIGHT-1), MIN_Y, MAX_Y)
    x = y = 0
    for iter in range(max_iter):
        xx = x*x
        yy = y*y
        t_x = xx - yy + a
        t_y = (x+x)*y + b
        x = t_x
        y = t_y
        if xx+yy>4:
            arr[j, i] = iter+1 -math.log(math.log(xx+yy)/math.log(2))
            return
    arr[j, i] = 0

@cuda.jit
def cuda_generate_orbit_trap_fractal(arr, max_iter):
    j, i = cuda.grid(2)
    a = scale(i/(WIDTH-1), MIN_X, MAX_X)
    b = scale(j/(HEIGHT-1), MIN_Y, MAX_Y)
    x = y = 0
    d = 1e9
    for _ in range(max_iter):
        xx = x*x
        yy = y*y
        t_x = xx - yy + a
        t_y = (x+x)*y + b
        x = t_x
        y = t_y
        # p1_x = 0.
        # p1_y = 0.25
        # p2_x = 0.
        # p2_y = -0.25
        # p3_x = .25
        # p3_y = 0
        # d = min(d, abs((x-p1_x)**2 + (y-p1_y)**2))
        # d = min(d, abs((x-p2_x)**2 + (y-p2_y)**2))
        # d = min(d, abs((x-p3_x)**2 + (y-p3_y)**2))
        d = min(d, abs(x))
    arr[j, i] = d

In [ ]:
# canvas = np.empty((HEIGHT, WIDTH), dtype=np.double)
# generate_fractal(canvas, escaped=escaped)
# px.imshow(np.log(canvas+1))

In [ ]:
canvas = cp.empty((HEIGHT, WIDTH), dtype=np.float32)
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(canvas.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(canvas.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
cuda_generate_fractal[blockspergrid, threadsperblock](canvas, 500)

w = WIDTH/1000
h = HEIGHT/1000
fig = plt.figure(frameon=False)
#fig.set_size_inches(w,h)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(np.log(cp.asnumpy(canvas+1)), cmap='inferno', aspect='auto')

plt.savefig("fractal1.png", dpi=1500)
print("Done!")

In [ ]:
canvas = cp.empty((HEIGHT, WIDTH), dtype=np.float32)
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(canvas.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(canvas.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
cuda_generate_orbit_trap_fractal[blockspergrid, threadsperblock](canvas, 3000)

fig = plt.figure(frameon=False)
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
glow = 0.0025
C2 = 1
out = cp.log(canvas+glow)
out = (out - out.min()) /(out.max()-out.min())
out = cp.log(out+1)
host_arr = cp.asnumpy(out)
ax.imshow(host_arr, cmap='plasma_r', aspect='auto')

plt.savefig("local_artifacts/fractal.png", dpi=5000)
print("Done!")

In [3]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 933120000

In [13]:
# Opens a image in RGB mode
im = Image.open(r"R:\other\fractals\local_artifacts\warble.png")
 
# Size of the image in pixels (size of original image)
# (This is not mandatory)
width, height = im.size
im1 = im.resize((2560, 1440))
im1.save("local_artifacts/warble_trim.png")
# Shows the image in image viewer
# im1.show()